In [1]:
import numpy as np
import matplotlib.pyplot as plt
from utils.notebook_config import *
import random
from utils.simulation import *

In [3]:
road = rna(L=30, tau=1, steps=10)

ribs = []

ribs.append(ribo(rna=road))

ribs[0].move(rna=road)
ribs[0].move(rna=road)
ribs[0].move(rna=road)

ribs.append(ribo(rna=road))

ribs[1].move(rna=road)
ribs[1].move(rna=road)
ribs[1].move(rna=road)
ribs[1].move(rna=road)
ribs[1].move(rna=road)

print(ribs[0].position())
print(road.occupation())

3
[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
